In [2]:
import pandas as pd
import numpy as np

In [18]:
import pandas as pd

# Read the HTML table into a list of DataFrames
dfs = pd.read_html(indiatable)

ImportError: lxml not found, please install it

In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Define the Wikipedia page URL
page_url = "https://en.wikipedia.org/wiki/List_of_Women's_Prize_for_Fiction_winners"

# Send a GET request to retrieve the page content
response = requests.get(page_url)
html_content = response.text
# Select the desired DataFrame (assuming it's the first one)
table_df = html_content

# Select the desired DataFrame (assuming it's the first one)
table_df = table_df[1]

# Process or display the DataFrame as needed
print(table_df)

!


In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Women's_Prize_for_Fiction_winners"

# Send a GET request to the URL and retrieve the content
response = requests.get(url)
content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(content, "html.parser")

# Find all the tables on the page
tables = soup.find_all("table", class_="wikitable")

# Specify the index of the table you want to extract data from
# table_index = 1

# # Select the specified table
# table = tables[table_index]

df = pd.DataFrame([])

for table in tables: 
    # Create the first column Year. I do this in this way because the column has merged cells
    # Extract the table rows
    rows = []
    for row in table.find_all("tr"):
        rows.append(row)

    # Create an empty list to store the values of the first column
    first_column = []
    # Print the DataFrame
    # Iterate over the rows and extract the year value from the <a> tag
    for row in rows:
        cells = row.find_all(["th", "td"])
        if cells:
            # Check if the first cell has rowspan attribute
            if cells[0].has_attr("rowspan"):
                # Get the year value from the <a> tag
                year = cells[0].find("a").text.strip()
                
                # Get the rowspan value
                rowspan = int(cells[0]["rowspan"])
                
                # Repeat the year value based on rowspan
                first_column.extend([year] * rowspan)


    # Create empty lists to store the column values
    second_column = []
    third_column = []
    fourth_column = []

    # Extract the data from the second, third, and fourth columns
    rows = table.find_all("tr")
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 4:
            second_column.append(cells[0].text.strip())
            third_column.append(cells[1].text.strip())
            fourth_column.append(cells[2].text.strip())

    # Create a DataFrame from the column values
    data = {
        "Year" : first_column, 
        "Author": second_column,
        "Title": third_column,
        "Result": fourth_column
    }
    datadf = pd.DataFrame(data)
    df = pd.concat([datadf, df])

# Print the DataFrame
df.Year = df.Year.astype(int)
df.to_parquet('assets/WPH.parquet')



In [76]:
use open library api to get book information only using title and author. Example: 

2023    7
2020    6
2001    6
1998    6
1997    6
1996    6
2009    6
2008    6
2007    6
2006    6
2005    6
2004    6
2003    6
2002    6
2000    6
2021    6
2019    6
2018    6
2017    6
2016    6
2015    6
2014    6
2013    6
2012    6
2011    6
2010    6
2022    6
1999    6
Name: Year, dtype: int64

In [11]:
import requests

# Set the base URL for the Open Library API
base_url = "https://openlibrary.org/"

# Set the endpoint for the API call
endpoint = f"api/books"

# Set the parameters for the API call
params = {"bibkeys": f"ISBN:{9780307593313}", "format": "json", "jscmd": "details"}

# Make the API call
response = requests.get(base_url + endpoint, params=params)

# Parse the JSON response
data = response.json()

# Check if the response contains data for the requested ISBN
if f"ISBN:{9780307593313}" in data:
    # Extract the key for the book from the response
    book_key = data[f"ISBN:{9780307593313}"]["key"]
    
    # Call the reviews endpoint to get the reviews for the book
    reviews_response = requests.get(base_url + f"{book_key}/reviews.json")
    
    # Parse the JSON response
    reviews_data = reviews_response.json()
    
    # Extract the reviews from the response
    reviews = reviews_data.get("reviews")

    # Calculate the average rating from the reviews
    if reviews:
        average_rating = sum([review["rating"] for review in reviews]) / len(reviews)
    else:
        average_rating = 0

    # Print the average rating
    print(f"Average rating: {average_rating}")
else:
    print("No results found.")


KeyError: 'key'

In [9]:
import requests

def get_book_info(book_name, author_name):
    base_url = 'http://openlibrary.org/search.json'
    params = {
        'title': book_name,
        'author': author_name
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data['numFound'] > 0:
        doc = data['docs'][0]  # Assuming the first result is the desired book
        title = doc['title']
        author = doc['author_name'][0] if 'author_name' in doc else 'Unknown Author'
        topic = doc['subject'][0] if 'subject' in doc else 'Unknown Topic'
        summary = doc['text'][0] if 'text' in doc else 'No summary available'
        average_rating = doc['average_rating'] if 'average_rating' in doc else 'No ratings available'

        print(f'Title: {title}')
        print(f'Author: {author}')
        print(f'Topic: {topic}')
        print(f'Summary: {summary}')
        print(f'Average Rating: {average_rating}')
    else:
        print('No book found with the given title and author.')

# Example usage
book_name = "The Great Gatsby"
author_name = "F. Scott Fitzgerald"
get_book_info(book_name, author_name)



Title: The Great Gatsby
Author: F. Scott Fitzgerald
Topic: Married people, fiction
Summary: No summary available
Average Rating: No ratings available


In [ ]:
Publish data
description 
ISBN
number of pages
goodreads id
average rating 

In [16]:
import requests

# Replace the ISBN with the ISBN of the book you're interested in
isbn = "9780307593313"

# Make a request to the Open Library API
response = requests.get(f"https://openlibrary.org/isbn/{isbn}.json")

# Extract the book information from the response
data = response.json()
data

{'works': [{'key': '/works/OL15029001W'}],
 'title': '1Q84',
 'publishers': ['Alfred A. Knopf'],
 'publish_date': '2011',
 'key': '/books/OL32895958M',
 'type': {'key': '/type/edition'},
 'identifiers': {},
 'isbn_13': ['9780307593313'],
 'ocaid': '1q840000mura_v3d3',
 'classifications': {},
 'languages': [{'key': '/languages/eng'}],
 'covers': [11526100],
 'lccn': ['2011014274'],
 'lc_classifications': ['PL856.U673 A61213 2011', 'PL856.U673A61213'],
 'source_records': ['ia:1q840000mura_v3d3', 'bwb:9780307593313'],
 'latest_revision': 5,
 'revision': 5,
 'created': {'type': '/type/datetime', 'value': '2021-07-29T12:16:13.463659'},
 'last_modified': {'type': '/type/datetime',
  'value': '2021-09-29T04:55:43.726733'}}

In [17]:
import requests

# Replace the ISBN with the ISBN of the book you're interested in
isbn = "9780307593313"

# Make a request to the Open Library API
response = requests.get(f"https://openlibrary.org/isbn/{isbn}.json")

# Extract the book information from the response
data = response.json()

# Access the details for the book
book_details = data.get(f"ISBN:{isbn}").get("details")

# Access the reviews for the book if they exist
book_reviews = book_details.get("reviews", [])

# Calculate the average rating for the book
if book_reviews:
    total_rating = sum(review["rating"] for review in book_reviews)
    average_rating = total_rating / len(book_reviews)
    print(f"Average rating: {average_rating:.2f}")
else:
    print("No reviews found for this book.")


AttributeError: 'NoneType' object has no attribute 'get'

In [1]:
import requests

isbn = "9780307593313"
base_url = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

# Make a request to the Google Books API
response = requests.get(base_url + isbn)

# Parse the response as JSON
data = response.json()

# Extract the rating from the book metadata
rating = data["items"][0]["volumeInfo"]["averageRating"]

# Print the rating
print(f"The rating for 1Q84 is {rating}")


The rating for 1Q84 is 3.5


In [19]:
data

{'error': 'notfound', 'key': '/works/OL15029001W/1Q84'}

In [23]:
import requests
import pandas as pd

def get_book_info(book_name, author_name):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'maxResults': 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'items' in data and len(data['items']) > 0:
        book = data['items'][0]['volumeInfo']
        title = book['title']
        authors = book['authors'] if 'authors' in book else ['Unknown Author']
        publish_date = book['publishedDate'] if 'publishedDate' in book else 'Unknown'
        description = book['description'] if 'description' in book else 'No description available'
        isbn = book['industryIdentifiers'][0]['identifier'] if 'industryIdentifiers' in book else 'Unknown'
        page_count = book['pageCount'] if 'pageCount' in book else 'Unknown'
        categories = book['categories'] if 'categories' in book else ['Unknown']
        average_rating = book['averageRating'] if 'averageRating' in book else 'Not rated'
        rating_count = book['ratingsCount'] if 'ratingsCount' in book else 'No ratings'
        language = book['language'] if 'language' in book else 'Unknown'

        book_info = pd.DataFrame({
            'Title': [title],
            'Author(s)': [", ".join(authors)],
            'Publish Date': [publish_date],
            'Description': [description],
            'ISBN': [isbn],
            'Page Count': [page_count],
            'Categories': [", ".join(categories)],
            'Average Rating': [average_rating],
            'Rating Count': [rating_count],
            'Language': [language]
        })

        return book_info
    else:
        print('No book found with the given title and author.')
        return pd.DataFrame()

# Example usage
book_name = "The Great Gatsby"
author_name = "F. Scott Fitzgerald"
book_info_df = get_book_info(book_name, author_name)

# Display the DataFrame
print(book_info_df)
book_info_df


              Title            Author(s) Publish Date   
0  The Great Gatsby  F. Scott Fitzgerald   2021-11-30  \

                                         Description           ISBN   
0  Hailed as one of the Great American Novels, Th...  9798200735792  \

   Page Count Categories Average Rating Rating Count Language  
0         270    Fiction      Not rated   No ratings       en  


,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language
0,The Great Gatsby,F. Scott Fitzgerald,2021-11-30,"Hailed as one of the Great American Novels, Th...",9798200735792,270,Fiction,Not rated,No ratings,en


In [87]:

def get_book_info(book_name, author_name):

    """
    Function to get book details using Google API. 
    Params: book_name, author_name
    
    """
    import requests
    import pandas as pd
    import numpy as np

    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'maxResults': 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'items' in data and len(data['items']) > 0:
        book = data['items'][0]['volumeInfo']
    else:
        # Adjust the book name with the added string ': A novel'
        adjusted_book_name = f"{book_name}: A novel"
        params['q'] = f'intitle:{adjusted_book_name}'

        response = requests.get(base_url, params=params)
        data = response.json()

        if 'items' in data and len(data['items']) > 0:
            book = data['items'][0]['volumeInfo']
        else:
            # Reset the query to check for just the title
            params['q'] = f'intitle:{book_name}'
            
            response = requests.get(base_url, params=params)
            data = response.json()
            
            if 'items' in data and len(data['items']) > 0:
                book = data['items'][0]['volumeInfo']
            else:
                book = {}

    # title = using the imput from the df to make sure the title return will match the df. 
    authors = book.get('authors', [np.nan])
    publish_date = book.get('publishedDate', np.nan)
    description = book.get('description', np.nan)
    identifiers = book.get('industryIdentifiers', [])
    isbn = identifiers[0]['identifier'] if identifiers else np.nan
    page_count = book.get('pageCount', np.nan)
    categories = book.get('categories', [np.nan])
    average_rating = book.get('averageRating', np.nan)
    rating_count = book.get('ratingsCount', np.nan)
    language = book.get('language', np.nan)

    book_info = pd.DataFrame({
        'Title': [book_name],
        'Author(s)': [", ".join(map(str, authors))],
        'Publish Date': [publish_date],
        'Description': [description],
        'ISBN': [isbn],
        'Page Count': [page_count],
        'Categories': [", ".join(map(str, categories))],
        'Average Rating': [average_rating],
        'Rating Count': [rating_count],
        'Language': [language]
    })

    return book_info

# Example usage
book_name = "half-blood blues"
author_name = "Esi Edugyan"
book_info_df = get_book_info(book_name, author_name)
book_info_df


,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language
0,half-blood blues,Esi Edugyan,2013-11-01,"The brilliant, bestselling, Giller Prize-winni...",1443433470,320,Blacks,3.5,619,en


In [91]:
df.query("Title.str.contains('Black Butterflies') == True")

,Year,Author,Title,Result
22,2023,Priscilla Morris,Black Butterflies,Finalist


In [74]:

def get_book_info(book_name, author_name):

    """
    Function to get book details using Google API. 
    Params: book_name, author_name
    
    """
    import requests
    import pandas as pd
    import numpy as np

    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'maxResults': 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if 'items' in data and len(data['items']) > 0:
        book = data['items'][0]['volumeInfo']
    else:
        # Adjust the book name with the added string ': A novel'
        adjusted_book_name = f"{book_name}: A novel"
        params['q'] = f'intitle:{adjusted_book_name}'

        response = requests.get(base_url, params=params)
        data = response.json()

        if 'items' in data and len(data['items']) > 0:
            book = data['items'][0]['volumeInfo']
        else:
            # Reset the query to check for just the title
            params['q'] = f'intitle:{book_name}'
            
            response = requests.get(base_url, params=params)
            data = response.json()
            
            if 'items' in data and len(data['items']) > 0:
                book = data['items'][0]['volumeInfo']
            else:
                book = {}

    title = book.get('title', np.nan)
    authors = book.get('authors', [np.nan])
    publish_date = book.get('publishedDate', np.nan)
    description = book.get('description', 'No description available')
    identifiers = book.get('industryIdentifiers', [])
    isbn = identifiers[0]['identifier'] if identifiers else 'Unknown'
    page_count = book.get('pageCount', np.nan)
    categories = book.get('categories', [np.nan])
    average_rating = book.get('averageRating', 'Not rated')
    rating_count = book.get('ratingsCount', 'No ratings')
    language = book.get('language', 'Unknown')

    book_info = pd.DataFrame({
        'Title': [title],
        'Author(s)': [", ".join(authors)],
        'Publish Date': [publish_date],
        'Description': [description],
        'ISBN': [isbn],
        'Page Count': [page_count],
        'Categories': [", ".join(categories)],
        'Average Rating': [average_rating],
        'Rating Count': [rating_count],
        'Language': [language]
    })

    return book_info

# Example usage
book_name = "The Mirror and the Light "
author_name = "Hilary Mantel"
book_info_df = get_book_info(book_name, author_name)
book_info_df


TypeError: sequence item 0: expected str instance, float found

In [88]:
def book_info_add(df): 
    """
    Iterates over a dataframe with book title and author to collect book information from Google Books API 
    by applying the get_book_info funcation to all the rows of books in the dataframe.
    Returns the book information in a dataframe
    Params: a dataframe with the columns Author and Title. 
    """

    # Create an empty DataFrame to store the book information
    combined_book_info = pd.DataFrame()

    # Iterate over the rows of the dataset
    for _, row in df.iterrows():
        book_name = row['Title']
        author_name = row['Author']
        
        # Call the get_book_info function for each book
        book_info = get_book_info(book_name, author_name)
        
        # Concatenate the resulting DataFrame to the combined_book_info DataFrame
        combined_book_info = pd.concat([combined_book_info, book_info])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)

    # Display the combined DataFrame
    return combined_book_info

,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language
0,Hamnet,Maggie O'Farrell,2020-07-21,NATIONAL BOOK CRITICS CIRCLE AWARD WINNER • NE...,9780525657613,320.0,Fiction,4.0,98.0,en
1,Dominicana,Angie Cruz,2019-09-03,A GOOD MORNING AMERICA BOOK CLUB PICK Shortlis...,9781250205926,381.0,Fiction,4.0,18.0,en
2,"Girl, Woman, Other",Bernardine Evaristo,2019-11-05,NATIONAL BESTSELLER WINNER OF THE BOOKER PRIZE...,9780802156990,464.0,Fiction,4.5,52.0,en
3,A Thousand Ships,Natalie Haynes,2019-05-02,Shortlisted for the Women's Prize for Fiction ...,9781509836222,352.0,Fiction,4.0,15.0,en
4,The Mirror and the Light,Hilary Mantel,2022-03-29,AN INTERNATIONAL BESTSELLER WINNER OF THE WALT...,1443413747,912.0,nan,4.5,42.0,en
...,...,...,...,...,...,...,...,...,...,...
164,The Leper's Companions,Julia Blackburn,2013-10-31,"To escape from her own sadness, a woman finds ...",9781448190904,224.0,Fiction,3.0,1.0,en
165,Visible Worlds,Marilyn Bowering,1999,Two brothers in 1930s Manitoba find themselves...,0006551130,308.0,Arctic regions,NaN,NaN,en
166,The Short History of a Prince,Jane Hamilton,2010-09-01,Walter McCloud is a boy with dreams unlike mos...,9780307764072,368.0,Fiction,NaN,NaN,en
167,The Poisonwood Bible,Barbara Kingsolver,2008-09-04,**NOW INCLUDING THE FIRST CHAPTER OF DEMON COP...,9780571246212,633.0,Fiction,4.0,285.0,en


In [53]:
combined_book_info.query("Title.str.contains('Grace Williams Says it Loud') == True")

,Title,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language


In [90]:
import re

# Function to remove square brackets and their contents using regex
def remove_square_brackets(text):
    return re.sub(r'\[.*?\]', '', text)

# Apply the function to the 'Title' column
df['Title'] = df['Title'].apply(remove_square_brackets)

# book_df = pd.merge(df.assign(Title= df['Title'].str.lower()),
#                      combined_book_info.assign(Title=combined_book_info['Title'].str.lower()),
#                      on='Title', how='outer')

book_df = pd.merge(df,
                     combined_book_info,
                     on='Title', how='outer')
missing = book_df.query("Description.isnull() ")
missing

,Year,Author,Title,Result,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language
22,2023,Priscilla Morris,Black Butterflies,Finalist,Priscilla Morris,2022-05,NaN,0715654594,0.0,Sarajevo (Bosnia and Hercegovina),NaN,NaN,en
50,2014,Chimamanda Ngozi Adichie,Americanah,Finalist,Chimamanda Ngozi Adichie,2015-06-25,NaN,359618598X,608.0,nan,NaN,NaN,de
82,2019,Anna Burns,Milkman,Finalist,Anna Burns,2019,NaN,8899911509,456.0,Fiction,NaN,NaN,it


In [95]:
book_df.loc[book_df['Author(s)'].isnull()==True]

,Year,Author,Title,Result,Author(s),Publish Date,Description,ISBN,Page Count,Categories,Average Rating,Rating Count,Language
